# BTC交易结构简介与构造

_2020-04-08 王亚飞_

前置知识：UTXO模型, 公私钥, WIF, PSBT

## 基本结构

![btc_overview](https://ae01.alicdn.com/kf/H9b8f12ec080945ee8364ba05ff6c5a69a.png)

## 交易字段

![fields](https://ae01.alicdn.com/kf/H329f3b8bea4847f6b9c39be287bd5ce33.png)

## 例子

In [64]:
import {display} from 'tslab';

display.html(`<style>
.wrap {
    overflow: wrap;
}
.version:hover {
    background: red;
}
.inputcount:hover{
    color: red;
}
.input > .txid:hover {
    background:#8f6f2f;
}
.input > .vout:hover {
    color:yellow;
}
.input > .scriptSigsize:hover {
    color: orange;
}
.input > .scriptSig:hover {
    background: green;
}
.input > .sequence:hover {
    background: blue;
}
.outputcount:hover {
    color: red
}

.output > .value:hover {
    background: #2ed41e;
}
.output > .scriptPubKeysize:hover {
    color: #d34e9d;
}
.output > .scriptPubKey:hover {
    background: #9d2e1c;
}
.locktime:hover {
    background: blue;
}
</style>
<pre><div class="wrap" style="background:#363636; color: white"><span class="version" alt="version">01000000</span><span class="inputcount">01</span><span class="input"><span class="txid">7967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc</span><span class="vout">01000000</span><span class="scriptSigsize">6a</span><span class="scriptSig">47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825</span><span class="sequence">ffffffff</span></span><span class="outputcount">01</span><span class="output"><span class="value">4baf210000000000</span><span class="scriptPubKeysize">19</span><span class="scriptPubKey">76a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac</span></span><span class="locktime">00000000</span></div></pre>`)

01000000 01 7967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc 01000000 6a 47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825 ffffffff 01 4baf210000000000 19 76a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac 00000000

![details](https://learnmeabitcoin.com/technical/images/transaction-data/transaction-data.gif)

## 交易与锁

在交易中我们常常会提到签名，实质上是提供一个解锁脚本使之对应在input中制定的vout的解锁脚本。简而言之，解锁的过程如下：

'message'_ori ---private key---> signature ---public key---> 'message'_dec

这样我们可以验证解码的消息是否和原始消息一致来验证公私钥是否匹配。（ps， 上图过程可逆， 同时也是非对称加密的基本过程，ssh, ssl等也建立于之上。）

## 交易类型

这里声明三个观念：
- private key：一个数字（通常会通过素性检验）
- public key: 一个和private key匹配的另一个数字（通常也会通过素性检验）
- address: public key的hash

下面列出一些常见的lockScript：
### Pay to Public key
```
04a9d6840fdd1497b3067b8066db783acf90bf42071a38fe2cf6d2d8a04835d0b5c45716d8d6012ab5d56c7824c39718f7bc7486d389cd0047f53785f9a63c0c9d OP_CHECKSIG
```
### Pay to Publib Key Hash
```
OP_DUP OP_HASH160 fde0a08625e327ba400644ad62d5c571d2eec3de OP_EQUALVERIFY OP_CHECKSIG
```

### Pay to MultiSig
```
OP_1 0378ee11c3fb97054877a809ce083db292b16d971bcdc6aa4c8f92087133729d8b 1283b5fbf5cc62d4399dfa1025c3e306295264494722c5085ceadadf1291f68125 a31752c9f17c628edc4c69c4c0846f8d814b21e046eabe06f9968a037ce0741c74 OP_3 OP_CHECKMULTISIG
```

### Pay to Script Hash
```
OP_HASH160 10b0ed2d1698ff0f0ea151cf41988d05b728746a OP_EQUAL
```

### Null Data
```
OP_RETURN 68656c6c6f20776f726c64
```

## The Stack
![Stack](https://ae05.alicdn.com/kf/H3f2b1181af9f41b291ebc530205531e4J.png)

In [1]:
import * as bitcoin from "bitcoinjs-lib"
import { ECPair, networks, Psbt } from "bitcoinjs-lib"
import { RegtestUtils } from "regtest-client"

In [2]:
const client = new RegtestUtils({APIURL: "http://localhost:18222/1"})

In [3]:
async function createPayment(wif: string) {
    const keypairs = ECPair.fromWIF(wif, networks.regtest)
    const p2pkh = bitcoin.payments.p2pkh({pubkey: keypairs.publicKey, network: networks.regtest})
    return {
        payment: p2pkh,
        keys: keypairs
    }
}

In [4]:
async function createInput(utils: RegtestUtils, sender: any, amount: number) {
    const unspent = await utils.faucetComplex(sender.output, amount)
    const utx = await utils.fetch(unspent.txId);
    const nonWitnessUtxo = Buffer.from(utx.txHex, 'hex');
    return {
     hash: unspent.txId,
     index: unspent.vout,
     nonWitnessUtxo
    };
}

In [5]:
const requester = await createPayment("93Dt9qVXNDyMcLR9Wq5QRS72pUtG1eQ1CtFLd36kM2qntvThQE3")

In [6]:
const input = await createInput(client, requester.payment, 2e6)
const data = bitcoin.payments.embed({data: [ Buffer.from("hello, world", "utf-8")]})

In [7]:
const tx = new Psbt({ network: networks.regtest});
tx.setVersion(2)
    .setLocktime(0)
    .addInput(input)
    .addOutput({
        address: "mhNzyD5ZdQZ38WSdRftnzE16msN96fmXpe",
        value: 2e4
    })
    .addOutput({
        script: data.output!,
        value: 0
    })
    .signAllInputs(requester.keys).validateSignaturesOfAllInputs();

true


In [8]:
const raw_tx = await tx.finalizeAllInputs().extractTransaction(true).toHex();

In [9]:
raw_tx

0200000001f4e145bf0b00ae902bd59219d7b4f9029797604cabe542bcea604d95361cfdd7000000008a47304402204b2e2302bc77f0ff95734775c95d44427f49b276aae2458d227cb61ff301812c0220070b8ff9357e1ecec9b09d507144126cf1a1df0d50c51731b66a2faaebdf92fb014104c226c854246153a52226ea043de19a32f2e40bdb8c7b347381d527a96eefa3dcbfdc874c6f4e83224320477b898d332b4d990a62d927163631420765ef448accffffffff02204e0000000000001976a914146f282c2297b9f5dcfe66897eccc74d29eed56188ac00000000000000000e6a0c68656c6c6f2c20776f726c6400000000


## Reference
- Greg Walk, Aug 2015, Transaction Data([https://learnmeabitcoin.com/technical/transaction-data](https://learnmeabitcoin.com/technical/transaction-data))
- Royal Fork, Nov 2014, Deconstructing Transactions([http://royalforkblog.github.io/2014/11/20/txn-demo/](http://royalforkblog.github.io/2014/11/20/txn-demo/))
- OpCodes ([https://en.bitcoin.it/wiki/Script#Constants](https://en.bitcoin.it/wiki/Script#Constants))